Nathaniel's Branch

In [1]:
# Initalize GPU
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/CAIS_Sentiment140_Project/
!unzip trainingandtestdata.zip
%ls

/content/drive/MyDrive/CAIS_Sentiment140_Project
Archive:  trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
testdata.manual.2009.06.14.csv             trainingandtestdata.zip
training.1600000.processed.noemoticon.csv


In [8]:
import pandas as pd
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text' ])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 1,600,000



In [9]:
# Display 10 random rows from the data.
df.sample(10)

,target,ids,date,flag,user,text
172,0,1467856426,Mon Apr 06 22:31:41 PDT 2009,NO_QUERY,AthenaA42,Grrr.. my ipods acting weird too! Jai ho and t...
995174,4,1835672522,Mon May 18 06:30:34 PDT 2009,NO_QUERY,feedmeshow,@lalunera Thought you might want to see that o...
1595843,4,2192514719,Tue Jun 16 07:09:04 PDT 2009,NO_QUERY,funkyblurooster,@mission2be thanks..trying to keep pace with t...
1050491,4,1960715393,Fri May 29 08:23:45 PDT 2009,NO_QUERY,kayteedeeaye,@Jsestan oh yes i hope so!! get the gang toget...
890812,4,1688858601,Sun May 03 12:04:54 PDT 2009,NO_QUERY,jdsalinger,@EverywhereTrip welcome to Rome! do send me a ...
1585261,4,2190656855,Tue Jun 16 03:21:28 PDT 2009,NO_QUERY,kismetfaith,finally.. got kismet on twitter - check!!
1446582,4,2062525532,Sat Jun 06 23:29:19 PDT 2009,NO_QUERY,angelambrandt,True love is never bliss 100%.. but when there...
1405786,4,2055244649,Sat Jun 06 08:58:13 PDT 2009,NO_QUERY,lucybones,http://twitpic.com/6qzjd - today I met... Mr @...
632969,0,2232852584,Thu Jun 18 20:29:10 PDT 2009,NO_QUERY,cpayan,"@bsoler Heh, I was wondering about that &quot;..."
177855,0,1965819425,Fri May 29 16:36:05 PDT 2009,NO_QUERY,AMF7,@lid86 he already got a shot at having his own...


In [24]:
print("0, negative sentiment")
df.loc[df.target == 0].sample(5)[['target', 'text']]

0, negative sentiment


,target,text
232746,0,"i wanna be with you forever, if tomorrows not ..."
506335,0,I'm so lost without you. I wake up reaching fo...
733938,0,going for buffet dinner when you're still full...
72761,0,School is way too early
774771,0,@lintably I think I am a knight sometimes..rus...


In [25]:
print("4, positive sentiment")
df.loc[df.target == 4].sample(5)[['target', 'text']]

4, positive sentiment


,target,text
1549338,4,@paperrelics ooo sounds like an awesome prize ...
1289704,4,"@MIKEIRV it was pretty decent, I can't really ..."
1244864,4,has decided to slowly build my work wardrobe b...
1500918,4,Trying to write a proposal at 3am &amp; I got ...
867888,4,has decided on dinner for the 16th ~ haha ~ bu...


In [26]:
#Extract text and target labels
texts = df.text.values
targets = df.target.values